### CS 421 PROJECT

Team members:
- Emmanuel Oh
- Lee Shiying
- Megan Tan
- Sheffield Lok

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

BATCHES_DIR = 'batches/'
PREDS_DIR = 'predictions/'

# Load data from previous batches as training sets
batch1=np.load(BATCHES_DIR+"first_batch.npz")
batch2=np.load(BATCHES_DIR+"second_batch_with_labels.npz")
batch3=np.load(BATCHES_DIR+"third_batch_with_labels.npz")
batch4=np.load(BATCHES_DIR+"fourth_batch_with_labels.npz")

In [ ]:
# Convert loaded data into pd.DataFrame objects
X1=pd.DataFrame(batch1["X"], columns=["user", "item", "rating"])
y1=pd.DataFrame(batch1["y"], columns=["user", "label"])
X2=pd.DataFrame(batch2["X"], columns=["user", "item", "rating"])
y2=pd.DataFrame(batch2["y"], columns=["user", "label"])
X3=pd.DataFrame(batch3["X"], columns=["user", "item", "rating"])
y3=pd.DataFrame(batch3["y"], columns=["user", "label"])
X4=pd.DataFrame(batch4["X"], columns=["user", "item", "rating"])
y4=pd.DataFrame(batch4["y"], columns=["user", "label"])


### Exploratory data analysis

In [ ]:
def show_metrics(t: str, X: pd.DataFrame):
    real = X.loc[X['label'] == 0]
    fake = X.loc[X['label'] == 1]
    r_real: pd.Series = real['rating']
    r_fake: pd.Series = fake['rating']
    metrics = pd.DataFrame(
        {
            'Status': ['Real', 'Fake'],
            'Count': [len(r_real), len(r_fake)],
            'Mean': [r_real.mean(), r_fake.mean()],
            'Median': [r_real.median(), r_fake.median()],
            'Std': [r_real.std(), r_fake.std()],
            'Min': [r_real.min(), r_fake.min()],
            'Max': [r_real.max(), r_fake.max()]
        }
    )
    print(f'------------------------------------{t}------------------------------------')

    print(metrics)

    bins = [0,1,2,3,4,5,6]
    fig, axs = plt.subplots(2)
    axs[0].hist(r_real, bins)
    axs[1].hist(r_fake, bins)
    plt.show()
    

In [ ]:
t1 = 'Batch 1'
X1_labeled: pd.DataFrame = pd.merge(X1, y1)
t2 = 'Batch 2'
X2_labeled: pd.DataFrame = pd.merge(X2, y2)
t3 = 'Batch 3'
X3_labeled: pd.DataFrame = pd.merge(X3, y3)
t4 = 'Batch 4'
X4_labeled: pd.DataFrame = pd.merge(X4, y4)

for t, X_labeled in [(t1, X1_labeled), (t2, X2_labeled), (t3, X3_labeled), (t4, X4_labeled)]:
    show_metrics(t, X_labeled)



### Feature engineering, model training and predictions

In [ ]:
# Concatenate all previous weeks' batches into a single dataframe each
data_X = [X1, X2, X3, X4]
data_y = [y1, y2, y3, y4]
X = pd.concat(data_X)
y = pd.concat(data_y)


y.drop('user', axis=1, inplace=True)

In [ ]:
# Define a function to compute useful features for model fitting
def get_features(X: pd.DataFrame):
  '''Takes in a DataFrame of user-item-rating entries and computes descriptive
  features based on each user's collated reviews


  Features include:
  - Review count
  - Total sum of reviews
  - Standard error of mean
  - Standard deviation
  - 25% quantile
  - 50% quantile
  - 75% quantile
  - mean
  - median
  - max
  - min
  - mode
  

  Returns a DataFrame with computed features for each unique user
  '''
  features = pd.DataFrame(X.groupby('user')['user'].max())
  features['count'] = X.groupby('user')['rating'].count()
  features['sum'] = X.groupby('user')['rating'].sum()
  features['sem'] = X.groupby('user')['rating'].sem()
  features['std'] = X.groupby('user')['rating'].std()
  features['25%'] = X.groupby('user')['rating'].quantile(0.25)
  features['50%'] = X.groupby('user')['rating'].quantile(0.50)
  features['75%'] = X.groupby('user')['rating'].quantile(0.75)
  features['mean'] = X.groupby('user')['rating'].mean()
  features['median'] = X.groupby('user')['rating'].median()
  features['max'] = X.groupby('user')['rating'].max()
  features['min'] = X.groupby('user')['rating'].min()
  features['mode'] = X.groupby('user')['rating'].apply(
      lambda x: x.value_counts().index[0]
    )
  return features


X = get_features(X).fillna(0)
X.head(10)


In [ ]:
from sklearn.model_selection import train_test_split


# Classification/Anomaly Detection methods
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import IsolationForest, RandomForestClassifier


# Model evaluation metrics
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, \
  recall_score, f1_score

In [ ]:
# Train-test split
X, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)


# Model-fitting
########### Logistic Regression ##########
lr = LogisticRegression()
lr.fit(X, np.ravel(y_train, order='C'))


########### K-Means ##########
# Set n_clusters to 1, as legitimate users should cluster around 1 distribution
N_Kmeans = 1
kmeans = KMeans(n_clusters=N_Kmeans, init='k-means++', max_iter=50, random_state=0)
kmeans.fit(X)

    
########## Gaussian Mixture ##########
# Set n_components to 1 as we treat normal users as a single cluster,
# with fake users as anomalies
N_GMix = 1
gm = GaussianMixture(n_components=N_GMix, random_state=0, covariance_type="full")
gm.fit(X)


########### Isolation Forest ##########
il=IsolationForest(n_estimators=500)
il.fit(X)


########## Random Forest ##########
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X, np.ravel(y_train, order='C'))

In [ ]:
# Predict on test set

scaler=[RobustScaler() for _ in range(N_Kmeans)]
dists_train, centres_train = kmeans.transform(X), kmeans.predict(X)
distnear_train = dists_train[range(centres_train.shape[0]), centres_train]
dists_test, centres_test = kmeans.transform(X_test), kmeans.predict(X_test)
distnear_test = dists_test[range(centres_test.shape[0]), centres_test]
for i in range(N_Kmeans):
    scaler[i].fit(distnear_train[centres_train==i].reshape(-1,1))
for i in range(N_Kmeans):
    distnear_test[centres_test==i] = scaler[i].transform(
        distnear_test[centres_test==i].reshape(-1,1)
    ).reshape(-1)


# For Kmeans and GM, normalize values to lie between 0-1, and take those 
# as predictions
def normalize(X):
  '''Normalizes values `i` in an array `X` to 0-1 by computing 
  `(i - min(x)) / (max(x) - min(x))`
  '''
  dist_min = min(X)
  dist_max = max(X)
  preds = np.array([(i-dist_min)/(dist_max-dist_min) for i in X])
  return preds


y_pred_lr = lr.predict_proba(X_test)[:,1]
y_pred_km = normalize(distnear_test)
y_pred_gm = normalize(-gm.score_samples(X_test))
y_pred_il = -il.score_samples(X_test)
y_pred_rf = rf.predict_proba(X_test)[:,1]

In [ ]:
labels = y_test['label']
y_pred_km_disc = np.round(y_pred_km)


print(f"F1: {f1_score(labels, y_pred_km_disc)}")
print(f"Precision: {precision_score(labels, y_pred_km_disc)}")
print(f"Recall: {recall_score(labels, y_pred_km_disc)}")

In [ ]:
def print_metrics(labels, pred):
    '''Takes in a set of true labels 0-1 and predictions (probabilistic
    or 0-1) and computes key evaluation metrics

    - AUC
    - Accuracy
    - Precision
    - Recall
    - F1 Score
    '''
    print(f"AUC         : {roc_auc_score(labels, pred)}")
    pred_discrete = np.round(pred)
    print(f"Accuracy    : {accuracy_score(labels, pred_discrete)}")
    print(f"Precision   : {precision_score(labels, pred_discrete)}")
    print(f"Recall      : {recall_score(labels, pred_discrete)}")
    print(f"F1          : {f1_score(labels, pred_discrete)}")


labels = y_test['label']
print("------------Logistic Regression------------")
print_metrics(labels, y_pred_lr)
print("------------K Means------------------------")
print_metrics(labels, y_pred_km)
print("------------Gaussian Mixture---------------")
print_metrics(labels, y_pred_gm)
print("------------Isolation Forest---------------")
print_metrics(labels, y_pred_il)
print("------------Random Forest------------------")
print_metrics(labels, y_pred_rf)

In [ ]:
# Final predictions
batch_final=np.load(BATCHES_DIR+"FINAL_batch.npz")
X_final=pd.DataFrame(batch_final["X"], columns=["user", "item", "rating"])
X_final_features = get_features(X_final)

In [ ]:
# Apply Random Forest classifier
y_pred_proba_rf = rf.predict_proba(X_final_features)[:,1]
predictions = pd.DataFrame(y_pred_proba_rf, columns=['y_pred'])
# Save output
np.savez(PREDS_DIR+'predictions_FINAL.npz', predictions)